In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/20 18:21:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
#df.printSchema()

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

In [3]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.ml import Pipeline 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer 
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexers = [
StringIndexer(inputCol="Dst IP", outputCol = "dstip_index"),
StringIndexer(inputCol="Dst Port", outputCol = "dstport_index"),
StringIndexer(inputCol="FIN Flag Cnt", outputCol = "fin_index"),
StringIndexer(inputCol="SYN Flag Cnt", outputCol = "syn_index"),
StringIndexer(inputCol="URG Flag Cnt", outputCol = "urg_index"),
StringIndexer(inputCol="PSH Flag Cnt", outputCol = "cwe_index"),
StringIndexer(inputCol="ECE Flag Cnt", outputCol = "ece_index"),
StringIndexer(inputCol="Ack Flag Cnt", outputCol = "ack_index"),
StringIndexer(inputCol="Label", outputCol = "label")]

In [4]:
pipeline = Pipeline(stages=indexers)
df = df.select("Dst IP", 
               "Dst Port",
               "FIN Flag Cnt", 
               "SYN Flag Cnt", 
               "URG Flag Cnt",
               "PSH Flag Cnt",
               "ECE Flag Cnt",
               "Ack Flag Cnt",
               "Label"
              )
indexed_rows_df = pipeline.fit(df).transform(df)

22/02/20 18:21:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
vectorAssembler = VectorAssembler(inputCols = ["dstip_index","dstport_index","fin_index", "syn_index", "urg_index", "cwe_index","ece_index", "ack_index"],outputCol = "features")
vindexed_rows_df = vectorAssembler.transform(indexed_rows_df)
#vindexed_rows_df.show(5, False)

In [6]:
splits = vindexed_rows_df.randomSplit([0.6,0.4], 42) 
# optional value 42 is seed for sampling 
train_df = splits[0] 
test_df = splits[1]

In [7]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train_df)

22/02/20 18:29:03 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:30:48 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:30:50 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:30:54 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:30:55 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/02/20 18:30:55 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/02/20 18:30:55 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:30:59 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:31:00 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:31:03 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:31:04 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
22/02/20 18:31:

In [8]:
predictions_df = lrModel.transform(test_df)

In [9]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy))
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
nbf1 = evaluator.evaluate(predictions_df) 
print("F1 score = " + str(nbf1))

22/02/20 18:33:35 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


Test accuracy = 0.9055975994071201


22/02/20 18:35:21 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


F1 score = 0.9055469891871994
